# LLM-powered AI Agents

Table of contents
1. Understanding LLMs
2. Tools
3. Chat-based AI Agents
4. Service-based AI agents
5. Multi-Agents

In [1]:
import json
import pandas as pd
import random
from pathlib import Path
from pydantic import BaseModel, Field
from typing import Any
from io import StringIO
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole
from language_models.tools.tool import Tool
from language_models.proxy_client import ProxyClient
from language_models.agent import Agent, OutputType, Chain, ChainAgentBlock, ChainToolBlock, ChainFilterBlock
from language_models.settings import settings
from tools.current_date import current_date
from tools.earthquake import count_earthquakes, query_earthquakes, USGeopoliticalSurveyEarthquakeAPI
from tools.forecasting import get_forecast, get_regions
from pprint import pprint

In [2]:
proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

## 1. Understanding LLMs

Understanding LLMs requires balancing algorithmic reasoning and human thought. Algorithmic reasoning is deterministic, producing consistent outcomes from identical inputs, unlike human thought, which is creative and subjective. LLMs lie between these extremes: they are fluent in natural language but do not truly understand what they say. They can execute algorithms but are limited in proficiency. Practically, executing algorithms involves using tools for desired outcomes. LLMs thus represent a hybrid, processing and generating text while relying on external software for algorithm execution.

LLMs are like new employees who need guidance and tools to perform well. They have potential and language skills but depend on external resources to execute tasks effectively. Providing necessary resources ensures their optimal performance, just as with new hires. Neglecting essential guidance or tools may require adjustments, akin to adapting to the needs of a new employee to ensure success.

Consider both system prompts and task prompts:

- **System prompts:** Set general behavior guidelines, such as the persona the LLM should adopt or how it should handle specific tasks and edge cases. For instance, instruct the LLM to check the date before other tasks or respond in a specific format.

- **Task prompts:** For simple queries, a direct question might suffice. For complex tasks, use structured prompts. For example, when classifying an IT ticket, provide clear details about the ticket, user, and date to ensure accurate handling.

To get the best results from LLMs, it's important to craft clear and effective prompts. Prompt engineering is an iterative process. Start with something simple and add more details later. Things to consider:

1. **Be specific:** Provide detailed information to help the LLM understand your query and give tailored responses.
2. **Ask clear questions:** Ask one question at a time to minimize confusion.
3. **Ask follow-up questions:** Clarify incomplete or unclear initial responses with rephrased queries or additional context.
4. **Use full sentences:** Provide context with clear and concise sentences.
5. **Provide examples:** Use examples to help the LLM understand your requirements and respond appropriately.

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-35-turbo",
    max_tokens=256,
    temperature=0.2,
)

The following cell defines a movie review prompt, calls GPT to analyze the sentiment of the review.

In [4]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!"""

output = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(output)

Sentiment: Positive


Below, we've adjusted the prompt to explicitly request a response indicating whether the sentiment of the movie review is positive or negative. The goal is to receive a concise sentiment analysis without additional text, catering to scenarios where we only need a binary classification of sentiment.

In [5]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!

Respond with positive or negative."""

output = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(output)

positive


We continue to prompt the model to provide a binary sentiment label for the given movie review. The review is positive, so the expected response should be 1 for positive sentiment. This setup is useful when we want to streamline the process of labeling data for sentiment analysis tasks.

In [6]:
system_prompt = """Take the following movie review and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative)."""

prompt = """Wow! This movie was incredible. The acting was superb,
and the plot kept me on the edge of my seat. I highly recommend it!"""

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(output)

1


Now, we present an unrelated question to the language model, asking about the weather in Seattle. However, we still provide the system prompt requesting a binary sentiment label for a movie review. This edge case might result in unexpected or irrelevant responses from the language model, demonstrating the importance of providing relevant prompts for accurate analysis.

In [7]:
system_prompt = """Take the following movie review determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative)."""

prompt = "Will it rain in Seattle today?"

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
pprint(output)

("I'm sorry, I am an AI language model and I do not have access to real-time "
 'weather information. I recommend checking a reliable weather website or '
 'using a weather app to get the most accurate and up-to-date forecast for '
 'Seattle.')


To cover edge cases, we provide clear instructions to the language model on how to handle scenarios where the input does not match the expected format. In this case we ask the LLM to respond with -1.

In [8]:
system_prompt = """Take the following movie review and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative).

If you don't receive a movie review, respond with -1."""

prompt = "Will it rain in Seattle today?"

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(output)

-1


## 2. Tools

LLMs use various tools to achieve specific goals, streamline operations, and automate tasks. These tools include:

1. **Data retrieval tools:** Extract information from systems or databases using APIs, SDKs, and real-time metrics.
2. **Communication tools:** Facilitate data exchange with external stakeholders via emails, notifications, or alerts.
3. **Data manipulation tools:** Update or modify data within systems, often requiring approval to manage operational impacts.

Specialized tools also exist to handle tasks LLMs struggle with, like performing calculations or accessing current date and time.

In [9]:
prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

output = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(output) # correct answer: $1,289.98

Total Raw Cost = $1,290.98


In the following code cell, we define a simple calculator function and a Pydantic - you can do it in a different way but for simplicity we will use Pydantic - model Calculator to represent its input arguments. We then create a tool instance calculator_tool using the Tool class from our repository, specifying the calculator function, its name, description, and the Pydantic model for argument validation.

By using Pydantic for JSON model schemas, we ensure that the LLM understands how to use the tools effectively. Additionally, Pydantic allows us to validate the LLM responses easily.

In [10]:
def calculator(expression: str) -> Any:
    return eval(expression)

class Calculator(BaseModel):
    expression: str = Field(description="A math expression")

calculator_tool = Tool(
    function=calculator,
    name="Calculator",
    description="Use this tool when you want to do calculations",
    args_schema=Calculator,
)

print(calculator_tool)

- Tool Name: Calculator, Tool Description: Use this tool when you want to do calculations, Tool Input: {'expression': {'description': 'A math expression', 'title': 'Expression', 'type': 'string'}}


In the system prompt below, we provide clear instructions to the language model on how to utilize the available tools, particularly the calculator tool, to calculate the result accurately based on the given prompt. This is essential for guiding the language model in effectively accessing and leveraging external tools to perform specific tasks, ensuring accurate and helpful responses to user queries.

In this case, we request JSON output from the language model, specifying that it should include a thought, tool, and tool input in its response. This structured format ensures clarity and consistency in the model's outputs, facilitating easy parsing and interpretation. While JSON output is suitable for this demonstration, in practical applications, text responses parsed via regex are often preferred for their simplicity and cost-effectiveness, as they eliminate the need to output JSON formatting elements such as brackets.

In [11]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools:
{calculator_tool}

Please ALWAYS use the following JSON format:
{{
  "thought": "Explain your thought. Consider previous and subsequent steps",
  "tool": "The tool to use. Must be on of {calculator_tool.name}",
  "tool_input": "Valid keyword arguments",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
output = json.loads(output)
print(json.dumps(output, indent=4))

{
    "thought": "To calculate the total raw cost, we need to add up all the individual costs.",
    "tool": "Calculator",
    "tool_input": {
        "expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"
    }
}


By using the input arguments the LLM provided in its response, we can successfully run software code and show the return value or observation to the LLM.

In [12]:
print(calculator(**output["tool_input"]))

1289.98


We introduce an observation to capture the intermediate steps or observations made during the calculation process. Given the model's inability to recall previous conversations, we introduce a mechanism to track and provide previous steps to the model. This ensures that the model can make informed decisions on when to provide the user with the final answer.

It's important to recognize that LLMs can sometimes produce errors in their output, such as generating incorrect JSON or invalid tool input arguments. To address this, it is crucial to highlight these errors to the LLM in hopes that it can correct them. However, the LLM may not always be able to fix its own mistakes. In such instances, after several iterations of feedback and corrections, we may need to provide a fallback response, such as "None" or empty strings.

This iterative process, combined with structured output formats and tool usage, forms the basis of an LLM-powered AI agent. While it may seem like magic in demonstrations, the underlying mechanism is actually quite straightforward. When using a library, keep in mind that the structured output format is generally managed by the library itself.

![ReAct prompting](../img/react.png)

In [13]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools:
{calculator_tool}

Please ALWAYS use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use. Must be on of {calculator_tool.name}",
  "tool_input": "Valid keyword arguments",
}}

Observation: tool result
... (this Thought/Tool/Tool input/Observation can repeat N times)

When you know the answer, you MUST use the following JSON format:
{{
  "thought": "I know what to respond",
  "tool": "Final Answer",
  "tool_input": "The final answer to the question",
}}"""

prompt = """Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12

This was your previous work:
Thought: The user wants me to calculate the total raw cost. I will use the Calculator tool.
Tool: Calculator
Tool input: {"expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"}
Observation: Tool response: 1289.98"""

output = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt),
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
output = json.loads(output, strict=False)
print(json.dumps(output, indent=4))

{
    "thought": "I know what to respond",
    "tool": "Calculator",
    "tool_input": "1289.98"
}


## 3. Chat-based AI Agents

AI agents are mainly used in two ways, with one key application being chat-based interactions. Users converse with LLMs for tasks in customer support, recruitment, production adjustments, sales forecasting, etc. This mirrors the intuitive way humans think about language.

### Earthquake

The following code snippet configures an LLM to specialize in answering earthquake-related questions by simulating the expertise of a United States Geological Survey (USGS) expert. It integrates various tools, including those for earthquake information and current date retrieval, to enhance its responses. Additionally, the LLM's output is formatted according to a specified schema, ensuring clarity and consistency in its responses. The structured output format of the final answer will become more useful later on.

In [14]:
current_date_tool = Tool(
    function=current_date,
    name="Current Date",
    description="Use this tool to access the current local date and time",
)

query_earthquakes_tool = Tool(
    function=query_earthquakes,
    name="Query Earthquakes",
    description="Use this tool to search recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
)

count_earthquakes_tool = Tool(
    function=count_earthquakes,
    name="Count Earthquakes",
    description="Use this tool to count and aggregate recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
)

In [15]:
system_prompt = "You are an United States Geological Survey expert who can answer questions regarding earthquakes."

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1000,
    temperature=0.2,
)

earthquake_agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{question}",
    prompt_variables=["question"],
    tools=[current_date_tool, count_earthquakes_tool, query_earthquakes_tool],
    output_type=OutputType.STRING,
    iterations=5,
)

The following question demonstrates how the LLM processes a user's query about the number of earthquakes that occurred on the current day. Using the provided tools, the LLM first retrieves the current date and time. This initial step is crucial for identifying and counting the seismic activities for the day. This process shows the power of [Chain-of-Thought](https://arxiv.org/abs/2201.11903), particularly the [ReAct](https://arxiv.org/abs/2210.03629) prompting method, which enables the AI to tackle multi-step problems. By breaking down the task into smaller, manageable steps — first obtaining the current date, then querying the relevant seismic data — the LLM can deliver correct answers.

In [16]:
output = earthquake_agent.invoke({"question": "How many earthquakes occurred today?"})

02/08/24 19:30:06 INFO Prompt:
How many earthquakes occurred today?
02/08/24 19:30:11 INFO Raw Output:
Thought: To answer this question, I need to count the number of earthquakes that occurred today. I will use the "Current Date" tool to get today's date and then use the "Count Earthquakes" tool with the start time set to the beginning of today and the end time set to the current time.

Tool: Current Date

Tool Input: {}
02/08/24 19:30:11 INFO Thought:
To answer this question, I need to count the number of earthquakes that occurred today. I will use the "Current Date" tool to get today's date and then use the "Count Earthquakes" tool with the start time set to the beginning of today and the end time set to the current time.
02/08/24 19:30:11 INFO Tool:
Current Date
02/08/24 19:30:11 INFO Tool Input:
{}
02/08/24 19:30:11 INFO Tool Response:
2024-08-02 19:30:11.066305
02/08/24 19:30:17 INFO Raw Output:
Thought: Now that I have the current date and time, I can use the "Count Earthquakes" 

In [17]:
print(output.final_answer)

There have been 153 earthquakes today.


Now we give the LLM a follow-up question where "Show me 3" refers to the earthquakes that occurred today. By keeping track of the chat history, the LLM can understand the context and continuity of the conversation. This enables the LLM to recognize that "today" is the time frame in question and to provide details about three specific earthquakes that occurred on the current day. This capability to maintain and utilize chat history is essential for  multi-turn interactions.

In [18]:
output = earthquake_agent.invoke({"question": "Show me 3."})

02/08/24 19:30:21 INFO Prompt:
Show me 3.
02/08/24 19:30:26 INFO Raw Output:
Thought: To show the details of 3 earthquakes that occurred today, I can use the "Query Earthquakes" tool with the same start and end times as before, but this time I will set the limit to 3.

Tool: Query Earthquakes

Tool Input: {"start_time": "2024-08-02T00:00:00", "end_time": "2024-08-02T19:30:11", "limit": 3}
02/08/24 19:30:26 INFO Thought:
To show the details of 3 earthquakes that occurred today, I can use the "Query Earthquakes" tool with the same start and end times as before, but this time I will set the limit to 3.
02/08/24 19:30:26 INFO Tool:
Query Earthquakes
02/08/24 19:30:26 INFO Tool Input:
{'start_time': '2024-08-02T00:00:00', 'end_time': '2024-08-02T19:30:11', 'limit': 3}
02/08/24 19:30:26 INFO Tool Response:
{'type': 'FeatureCollection', 'metadata': {'generated': 1722619826000, 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-08-02T00%3A00%3A00&endtime=202

In [19]:
print(output.final_answer)

Here are the details of 3 earthquakes that occurred today:

1. A magnitude 0.76 earthquake occurred 8 km NNW of The Geysers, CA. More details can be found [here](https://earthquake.usgs.gov/earthquakes/eventpage/nc75042777).

2. A magnitude 0.72 earthquake occurred 8 km NNW of The Geysers, CA. More details can be found [here](https://earthquake.usgs.gov/earthquakes/eventpage/nc75042767).

3. A magnitude 2.04 earthquake occurred 7 km NE of Antioch, CA. More details can be found [here](https://earthquake.usgs.gov/earthquakes/eventpage/nc75042762).


The LLM can also handle straightforward informational questions that don't require the use of additional tools. In this case, the question about the possibility of MegaQuakes (magnitude 10 or larger) can be answered directly by the language model without invoking any external tools. This shows the LLM's ability to recognize when it can rely on its internal knowledge base to provide an accurate response.

In [20]:
output = earthquake_agent.invoke({"question": "Can MegaQuakes really happen? Like a magnitude 10 or larger?"})

02/08/24 19:30:41 INFO Prompt:
Can MegaQuakes really happen? Like a magnitude 10 or larger?
02/08/24 19:30:50 INFO Raw Output:
Thought: The user is asking about the possibility of a magnitude 10 or larger earthquake, often referred to as a "MegaQuake". This is a question about the theoretical limits of earthquake magnitude, which doesn't require the use of any tools. 

Final Answer: In theory, an earthquake could reach a magnitude of 10 on the Richter scale. However, in practice, such an event is extremely unlikely. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960. A magnitude 10 earthquake would release 31.6 times more energy than the 1960 Chilean earthquake. The Earth's crust would likely not be able to accumulate the amount of stress needed to produce a magnitude 10 earthquake. Additionally, such an earthquake would cause catastrophic damage and loss of life.
02/08/24 19:30:50 INFO Thought:
The user is asking about the possibility of a magnitude 10 or larger

In [21]:
pprint(output.final_answer)

('In theory, an earthquake could reach a magnitude of 10 on the Richter scale. '
 'However, in practice, such an event is extremely unlikely. The largest '
 'earthquake ever recorded was a magnitude 9.5 in Chile in 1960. A magnitude '
 '10 earthquake would release 31.6 times more energy than the 1960 Chilean '
 "earthquake. The Earth's crust would likely not be able to accumulate the "
 'amount of stress needed to produce a magnitude 10 earthquake. Additionally, '
 'such an earthquake would cause catastrophic damage and loss of life.')


## 4. Service-based AI Agents

Alternatively, LLMs can be integrated into existing applications as services. For instance, a dashboard button could trigger an LLM to generate sales reports. Unlike traditional APIs, LLMs offer data interpretation and content generation, providing insights and actionable recommendations. This involves encapsulating the AI agent within a function that processes user requests and linking it to an API endpoint. Additionally this allows us to automate workflows based on events.

Almost everything in web or mobile applications involves text in some form. This text can be sent to a language model, which can then use the information to perform tasks such as recommending or comparing selected cars. Like chat-based agents, you can equip LLMs with tools to enhance their functionality.

### Contract Drafting

When it comes to using LLMs to draft contracts, it's important to consider that contracts are often lengthy documents, sometimes spanning over 100 pages. This renders conversation-based applications impractical, as LLMs cannot generate such extensive documents, and users prefer not to interact directly with the LLM and manage its outputs themselves.

Here's a potential approach for an application to draft contracts: Rather than expecting the LLM to generate the entire document at once, we can break it down into manageable sections. Users could provide bullet points for each section, allowing the LLM to formulate the corresponding paragraphs and suggest a title for each section. The application would then concatenate these outputs to create a document. Additionally, in practical scenarios, it's likely necessary to grant the LLM access to specific laws or legal references in some manner.

In [22]:
system_prompt = """You are a corporate lawyer. Take the follow bullet points and generate a draft of a section for a contract. Make it lengthy."""

task_prompt = """Bullet points:
{section}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1000,
    temperature=0.2,
)

class ContractSection(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")

contract_drafting_agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt=task_prompt,
    prompt_variables=["section", "section_name"],
    tools=None,
    output_type=OutputType.OBJECT,
    output_schema=ContractSection,
    iterations=3,
)

With the structured output format outlined earlier, the language model will provide us with both a title and the content for each section. Using the capabilities of structured output formats, we aren't restricted to requesting solely string outputs as the final answer; instead, we can ask for more complex data structures.

In [23]:
def generate_contract(contract_sections: list[str]) -> str:
    sections = []
    for contract_section in contract_sections:
        output = contract_drafting_agent.invoke({"section": contract_section})
        section = str(output.final_answer.title) + "\n\n" + str(output.final_answer.content)
        sections.append(section)
        contract_drafting_agent.chat.reset()
    return "\n\n".join(sections)

In [24]:
definitions = "Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA."

invoicing = """INVOICING AND PAYMENT TERMS
Clause 12.1(ii) of the GMA shall be cancelled and substituted as follow:
[*****]
[*****]
[*****]

Any other provision of Clause 12 shall remain in full force and effect."""

price_conditions = """PRICE CONDITIONS
(i) Clause 3.2 of the Exhibit 14 of the GMA shall be cancelled and substituted as follow:
“3.2 Technical conditions for prices adjustment
The prices set out in this Exhibit 14 shall be modified every [*****] at the occasion of the invoicing reconciliation pursuant to Clause 11
(“Reconciliation”) if the Standard Operations of the Aircraft, analyzed at the time of the adjustment (all calculations are made with figures corresponding to [*****], change by more or less
[*****] with respect to the estimated values of the same parameters, considered at the time of commencement of the Term.
As from the date this Agreement enters into force, the Parties agree to take into account the following basic operating parameters (the
“Standard Operations”) as a reference for the above calculation:
[*****]
[*****]
[*****]"""

amendment = "Amendment is effective starting on the date of its signature by both Parties."

confidentiality = """Confidential Information released by either of the Parties (the “Disclosing Party”) to the
other Party (the “Receiving Party”) shall not be released in whole or in part to any third party:
- Not to deliver, disclose or publish it to any third party including subsidiary companies and companies having an interest in its capital
- Use Confidential Information solely for the purpose of this Amendment
- Disclose the Confidential Information only to those of its direct employees
- Not to duplicate the Confidential Information nor to copy

Any Confidential Information shall remain the property of the Disclosing Party.

The Receiving Party hereby acknowledges and recognises that Confidential Information is protected by copyright Laws and related
international treaty provisions, as the case may be.

This shall survive termination or expiry of this Amendment for a period of five (5) years following such End Date."""

governing_law = """Pursuant to and in accordance with Section 5-1401 of the New York General Obligations Law.

Arbitration: in the event of a dispute arising out of or relating to this Amendment, including without limitation disputes regarding the
existence, validity or termination of this Amendment (a “Dispute”), either Party may notify such Dispute to the other through service of a
written notice (the “Notice of Dispute”).

Arbitration, and any proceedings, and meetings incidental to or related to the arbitration process, shall take place in New York.

Arbitration shall be kept confidential and the existence of the proceeding and any element.

During any period of negotiation or arbitration, the Parties shall continue to meet their respective obligations.

Notwithstanding any provision of this the Parties may, at any time, seek and decide to settle a Dispute.

Judgment upon any award may be entered in any court having jurisdiction.

Recourse to jurisdictions is expressly excluded except as provided for in the ICC Rules of Conciliation and Arbitration."""

miscellaneous = """Amendment contains the entire agreement between the Parties regarding the subject-matter.

Amendment shall not be varied or modified except by a written document duly signed."""

contract = generate_contract(
    contract_sections=[
        definitions,
        invoicing,
        price_conditions,
        amendment,
        confidentiality,
        governing_law,
        miscellaneous,
    ]
)

02/08/24 19:30:50 INFO Prompt:
Bullet points:
Capitalised terms, singular or plural, used in this Amendment, shall have the same meaning in the GMA.
02/08/24 19:30:59 INFO Raw Output:
Thought: The user wants to create a section in a contract that clarifies the usage of capitalized terms and their meanings in the context of an amendment and the General Master Agreement (GMA). The section should explain that the capitalized terms, whether used in singular or plural form, will have the same meaning as defined in the GMA. 

Final Answer: 
{
  "title": "Definition and Interpretation of Capitalised Terms",
  "content": "For the purposes of this Amendment, all terms that are capitalised, whether they are used in singular or plural form, shall have the same meaning as those defined in the General Master Agreement (GMA). This applies irrespective of whether the terms are used in the context of this Amendment or the GMA. The intention is to maintain consistency and avoid any potential ambiguity 

In [25]:
pprint(contract)

('Definition and Interpretation of Capitalised Terms\n'
 '\n'
 'For the purposes of this Amendment, all terms that are capitalised, whether '
 'they are used in singular or plural form, shall have the same meaning as '
 'those defined in the General Master Agreement (GMA). This applies '
 'irrespective of whether the terms are used in the context of this Amendment '
 'or the GMA. The intention is to maintain consistency and avoid any potential '
 'ambiguity or misunderstanding that may arise from the use of such terms. '
 'This provision is to ensure that the interpretation of the terms remains '
 'consistent throughout the entirety of the contractual documents, thereby '
 'preserving the integrity and intent of the original agreement.\n'
 '\n'
 'Amendment to Invoicing and Payment Terms\n'
 '\n'
 'Pursuant to the agreement between the parties, it is hereby agreed and '
 'acknowledged that Clause 12.1(ii) of the General Master Agreement (GMA) '
 'shall be cancelled and substituted with 

### Sentiment Analysis

For straightforward classification tasks, such as analyzing the sentiment of tweets, we can delegate the labeling process to the LLM. What's neat about this approach is that we can store the LLM's reasoning along with the sentiment label, facilitating the development of explainable AI systems.


In this scenario, it's important to note that this isn't a chat-based application where users manually feed tweets to the LLM one by one and record the results themselves. Instead, the process involves automating the sentiment analysis task, where the LLM classifies the sentiment of tweets without direct user intervention.

In [26]:
df_tweets = pd.read_csv("./data/tweets.csv.gz", compression="gzip", encoding="latin-1", names=["sentiment", "id", "date", "query", "user", "tweet"])
df_tweets = df_tweets.dropna()
df_tweets = df_tweets.where(df_tweets.sentiment != 2)
df_tweets["sentiment"] = df_tweets["sentiment"].map({4: 1, 0: 0})
df_tweets_sampled = df_tweets.sample(n=10)
df_tweets_sampled.head(10)

,sentiment,id,date,query,user,tweet
960085,1,1826277343,Sun May 17 08:07:26 PDT 2009,NO_QUERY,findingino,Had a great time last night
1275660,1,2001112658,Mon Jun 01 23:52:46 PDT 2009,NO_QUERY,rss2,@lauraulloa http://twitpic.com/6g89e - [...] I...
1139165,1,1976905806,Sat May 30 19:30:43 PDT 2009,NO_QUERY,chinatheblack,Movie night!!!! With him down stairs!!! &q...
1452923,1,2063182609,Sun Jun 07 01:40:14 PDT 2009,NO_QUERY,feiticeira,@christophermc Those pics from your prom are p...
1133301,1,1976153177,Sat May 30 16:42:10 PDT 2009,NO_QUERY,Haiku575,@pirateraina you must have eaten some bad alco...
301105,0,1998534365,Mon Jun 01 18:42:34 PDT 2009,NO_QUERY,art_and_coffee,I'm just about to have a siesta.. at 8:42 pm x...
1387831,1,2052965356,Sat Jun 06 02:44:02 PDT 2009,NO_QUERY,angelobott,working on a remix for France
1207829,1,1988748248,Sun May 31 23:15:29 PDT 2009,NO_QUERY,Pink_Hollyhock,- You think you know people. Then they tell yo...
1112115,1,1972348444,Sat May 30 09:03:25 PDT 2009,NO_QUERY,haileysweet,Yesterday evening and night was some good stuff
1317061,1,2014222878,Wed Jun 03 00:50:07 PDT 2009,NO_QUERY,roflucy,"@stephaaaaaniex i know, it's annoying also"


In [27]:
system_prompt = """Take the following tweet and determine the sentiment of the review.

Respond with 1 (positive) or 0 (negative).

If you don't receive a tweet, respond with -1."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=250,
    temperature=0.2,
)

class Tweet(BaseModel):
    reason: str = Field(description="The reason why you chose the sentiment")
    sentiment: int = Field(description="The sentiment of the tweet")

sentiment_analysis_agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="Tweet:\n{tweet}",
    prompt_variables=["tweet"],
    tools=None,
    output_type=OutputType.OBJECT,
    output_schema=Tweet,
    iterations=3,
)

In [28]:
def classify_sentiment(row) -> pd.Series:
    output = sentiment_analysis_agent.invoke({'tweet': row.tweet})
    reason = output.final_answer.reason or ""
    sentiment = output.final_answer.sentiment or 0
    sentiment_analysis_agent.chat.reset()
    return pd.Series([sentiment, reason], index=['prediction', 'reason'])

In [29]:
df_tweets_sampled[['prediction', 'reason']] = df_tweets_sampled.apply(classify_sentiment, axis=1)

02/08/24 19:32:16 INFO Prompt:
Tweet:
Had a great time last night 
02/08/24 19:32:19 INFO Raw Output:
Thought: The tweet expresses a positive sentiment as the user mentioned having a 'great time'. 

Final Answer: {'reason': 'The tweet expresses a positive experience', 'sentiment': 1}
02/08/24 19:32:19 INFO Thought:
The tweet expresses a positive sentiment as the user mentioned having a 'great time'.
02/08/24 19:32:19 INFO Final Answer:
reason='The tweet expresses a positive experience' sentiment=1
02/08/24 19:32:19 INFO Prompt:
Tweet:
@lauraulloa http://twitpic.com/6g89e - [...] It's clear to see, you're the reason that God made the girl. [...]  
02/08/24 19:32:24 INFO Raw Output:
Thought: The tweet seems to be complimenting the user @lauraulloa, implying that she is so special that she is the reason God made girls. This is a positive sentiment.

Final Answer: {'reason': 'The tweet is complimenting the user, indicating a positive sentiment.', 'sentiment': 1}
02/08/24 19:32:24 INFO Thou

In [30]:
df_tweets_sampled.head(10)

,sentiment,id,date,query,user,tweet,prediction,reason
960085,1,1826277343,Sun May 17 08:07:26 PDT 2009,NO_QUERY,findingino,Had a great time last night,1,The tweet expresses a positive experience
1275660,1,2001112658,Mon Jun 01 23:52:46 PDT 2009,NO_QUERY,rss2,@lauraulloa http://twitpic.com/6g89e - [...] I...,1,"The tweet is complimenting the user, indicatin..."
1139165,1,1976905806,Sat May 30 19:30:43 PDT 2009,NO_QUERY,chinatheblack,Movie night!!!! With him down stairs!!! &q...,1,The tweet expresses excitement and positive se...
1452923,1,2063182609,Sun Jun 07 01:40:14 PDT 2009,NO_QUERY,feiticeira,@christophermc Those pics from your prom are p...,1,The tweet is expressing a positive sentiment t...
1133301,1,1976153177,Sat May 30 16:42:10 PDT 2009,NO_QUERY,Haiku575,@pirateraina you must have eaten some bad alco...,-1,The tweet is more of a personal interaction ra...
301105,0,1998534365,Mon Jun 01 18:42:34 PDT 2009,NO_QUERY,art_and_coffee,I'm just about to have a siesta.. at 8:42 pm x...,0,The tweet is a neutral statement about the use...
1387831,1,2052965356,Sat Jun 06 02:44:02 PDT 2009,NO_QUERY,angelobott,working on a remix for France,-1,The tweet is just stating a fact without expre...
1207829,1,1988748248,Sun May 31 23:15:29 PDT 2009,NO_QUERY,Pink_Hollyhock,- You think you know people. Then they tell yo...,0,The tweet is more of a neutral statement or ob...
1112115,1,1972348444,Sat May 30 09:03:25 PDT 2009,NO_QUERY,haileysweet,Yesterday evening and night was some good stuff,1,"The user describes their experience as ""some g..."
1317061,1,2014222878,Wed Jun 03 00:50:07 PDT 2009,NO_QUERY,roflucy,"@stephaaaaaniex i know, it's annoying also",0,"The tweet expresses a feeling of annoyance, wh..."


### Structuring Unstructured Data

An excellent application of LLMs involves organizing unstructured data, such as text documents. Take, for instance, a collection of job descriptions. While there are various methods to tackle this task, like coding a parser or using optical character recognition, we opt to leverage an LLM for the job. Initially, we define the specific information we wish to extract from the job postings, such as the job title, salary, application instructions, and more. The LLM then looks at each job description and extracts the details for us. Subsequently, we can effortlessly store this dataset as a CSV file for further analysis and use.

In [31]:
path = Path("./data/jobs")
filenames = [file.name for file in path.iterdir() if file.is_file()]
filenames = random.sample(filenames, 5)

jobs = []
for filename in filenames:
    file_path = path / filename
    with open(file_path, "r", encoding="utf-8", errors="replace") as file:
        content = file.read()
        jobs.append(content)

With structured output formats, we can now request even more complex data structures. Here, we aim to retrieve strings, integers, and even lists containing strings. Leveraging Pydantic, we can validate the LLM's output. As shown by the dataset below, we can see that the LLM is capable of handling even more complex data structures.

In [32]:
system_prompt = """Take the following job and extract data about the job.

Respond with the job information:
- job title: title of the job.
- job class no: class number.
- job duties: duties of the job.
- open date: when the position was created. Use DD-MM-YYYY.
- salary: the salary ranges.
- deadline: when the application deadline is. Use DD-MM-YYYY.
- application form: online or email or fax.
- where to apply: url or location."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=500,
    temperature=0.2,
)

class Job(BaseModel):
    job_title: str = Field(description="The job title.")
    job_class_no: int = Field(description="The job class number.")
    job_duties: str = Field(description="The duties of the job.")
    open_date: str = Field(description="When the position was opened. Format: DD-MM-YYYY.")
    salary: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")
    deadline: str = Field(description="The application deadline. Format: DD-MM-YYYY")
    application_form: str = Field(description="The form of the application (e.g. online, fax, email).")
    where_to_apply: str = Field(description="The url to apply at or location to send the fax or email address.")

job_data_agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="Job description:\n{job}",
    prompt_variables=["job"],
    tools=None,
    output_type=OutputType.STRUCT,
    output_schema=Job,
    iterations=3,
)

In [33]:
def extract_jobs(jobs: list[str]) -> pd.DataFrame:
    data = []
    for job in jobs:
        output = job_data_agent.invoke({"job": job})
        data.append(output.final_answer)
        job_data_agent.chat.reset()
    return pd.DataFrame(data)

In [34]:
df_jobs = extract_jobs(jobs)

02/08/24 19:33:01 INFO Prompt:
Job description:
PLUMBER SUPERVISOR

Class Code:      3446
Open Date:  10-05-18
(Exam Open to Current City Employees)

ANNUAL SALARY

$108,482 (flat-rated)
The salary in the Department of Water and Power is $115,048 (flat-rated).

NOTES:

1. For information regarding reciprocity between the City of Los Angeles departments and LADWP, go to http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.
2. Annual salary is at the start of the pay range. The current salary is subject to change. Please confirm the starting salary with the hiring department before accepting a job offer.

DUTIES

A Plumber Supervisor assigns, reviews and evaluates the work of Senior Plumbers, Plumbers and helpers engaged in the layout, installation, repair, and maintenance of sanitary and industrial plumbing systems, process piping and fuel dispensing systems, including air, gas, water, steam, industrial waste, gasoline and oil pipelines; determines procedures and methods to use in in

In [35]:
df_jobs.head()

,job_title,job_class_no,job_duties,open_date,salary,deadline,application_form,where_to_apply
0,PLUMBER SUPERVISOR,3446,"A Plumber Supervisor assigns, reviews and eval...",10-05-2018,"[$108,482, $115,048]",18-10-2018,online,https://www.governmentjobs.com/careers/lacity/...
1,COMMUNICATIONS ENGINEERING ASSOCIATE,7607,A Communications Engineering Associate perform...,08-10-2018,"[$66,231 to $96,841, $74,082 to $108,346, $82,...",23-08-2018,online,https://www.governmentjobs.com/careers/lacity
2,PRINCIPAL CIVIL ENGINEERING DRAFTING TECHNICIAN,7219,A Principal Civil Engineering Drafting Technic...,11-02-2018,"[$67,964 to $99,347, $97,071 to $120,602 and $...",15-11-2018,online,https://www.governmentjobs.com/careers/lacity/...
3,SHIFT SUPERINTENDENT WASTEWATER TREATMENT,7242,A Shift Superintendent Wastewater Treatment su...,24-07-2015,"[$96,444 to $119,830, $101,810 to $126,491]",06-08-2015,online,http://agency.governmentjobs.com/lacity/defaul...
4,STREET SERVICES WORKER,4150,A Street Services Worker rakes hot mix asphalt...,23-03-2018,"[$41,760 to $61,053, $44,411 to $64,915, $46,8...",05-04-2018,online,https://www.governmentjobs.com/careers/lacity


## 5. Multi-Agents

When tasks assigned to an LLM become too complex, it is essential to divide the AI agent into multiple components. The common approach is to split the AI agent into multiple agents with specific goals, either sequentially chaining them or linking them in a graph-like structure. This is one way to let multiple LLMs collaborate to solve a specific problem.

### Comparing Unstructured Data

When comparing two job descriptions, one approach involves presenting both job postings to an LLM to identify similarities and differences. However, this method may not yield optimal results, as job descriptions often include non-essential information such as "equal employment opportunity" statements. To address this, we can deconstruct the problem by initially tasking an LLM to extract relevant data from each job description. In our case 2 instances as we are comparing 2 jobs. Subsequently, this condensed information can be inputted into a 3rd LLM, tasked with analyzing the extracted data to identify differences and similarities between the two jobs.

The structured format of the input and output fields in the graph enables us to reference data across the entire chain. This flexibility allows us to control exactly what each LLM processes. By tailoring the information each LLM receives, we ensure they have the essential data to solve their specific task, while also preventing unnecessary information. This approach not only reduces costs but also safeguards against misleading inputs that could compromise the quality of the LLM outputs.

However, a concern may arise if an LLM fails to provide the expected output, potentially breaking the entire workflow from a software perspective. As mentioned previously, in the event of a missing output, we use a default response. In this example walkthrough, we output "None" for all fields. Consequently, we pass "None" to all subsequent LLMs, ensuring they can still execute without encountering errors. However, it's important to note that the final output of the chain could also be "None" since the subsequent LLMs won't be able to solve the problem without the necessary input or the LLM could prompt the user to provide more information, telling the user to try again. 
Additionally, you can equip LLMs with tools to enhance their functionality, allowing each model to solve multi-step subproblems before passing the results to the next model in the chain.

| Input |
|------|
| job 1 |
| job 2 |

<br/>

| LLM that extracts job data |
|------|
| `Input` job 1 |
| `Output` job title 1, job duties 1, salary 1 |

<br/>

| LLM that extracts job data |
|------|
| `Input` job 2 |
| `Output` job title 2, job duties 2, salary 2 |

<br/>

| LLM that compares 2 jobs |
|------|
| `Input` job title 1, job duties 1, salary 1, job title 2, job duties 2, salary 2 |
| `Output` differences, similarities |

<br/>

| Output |
|------|
| differences |
| similarities |

In [36]:
def get_job(path: str) -> str:
    with open(path, "r", encoding="utf-8") as file:
        content = file.read()
        return content

job1 = get_job("./data/jobs/ELECTRICAL ENGINEERING ASSOCIATE 7525 093016 REV 100416.txt")
job2 = get_job("./data/jobs/ELECTRICAL MECHANIC 3841 012017.txt")

In [37]:
system_prompt = "Take the following job and extract data about the job"

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=500,
    temperature=0.2,
)

class Job(BaseModel):
    title: str = Field(description="The job title.")
    duties: str = Field(description="The duties of the job.")
    salary: list[str] = Field(description="A list of salary ranges. Format: 'min salary to max salary'.")

job_agent1 = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="Job description:\n{job1}",
    prompt_variables=["job1"],
    tools=None,
    output_type=OutputType.OBJECT,
    output_schema=Job,
    iterations=3,
)

job_agent2 = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="Job description:\n{job2}",
    prompt_variables=["job2"],
    tools=None,
    output_type=OutputType.OBJECT,
    output_schema=Job,
    iterations=3,
)

In [38]:
system_prompt = "Take the following 2 job descriptions and respond with the similarities and differences of the jobs."

task_prompt = """Compare the 2 given job descriptions:

Job 1:
Job title: {job1.title}
Job duties:
{job1.duties}
Salary:
{job1.salary}

Job 2:
Job title: {job2.title}
Job duties:
{job2.duties}
Salary:
{job2.salary}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=500,
    temperature=0.2,
)

class JobComparison(BaseModel):
    similarities: str = Field(description="The job similarities.")
    differences: str = Field(description="The job differences.")

job_comparison_agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt=task_prompt,
    prompt_variables=["job1", "job2"],
    tools=None,
    output_type=OutputType.OBJECT,
    output_schema=JobComparison,
    iterations=3,
)

In [39]:
class CompareJobs(BaseModel):
    job1: str = Field(description="The first job")
    job2: str = Field(description="The second job")

chain = Chain(
    name="Compare Jobs",
    description="Allows you to compare key information of 2 jobs",
    inputs=CompareJobs,
    output="job_comparison",
    blocks=[
        ChainAgentBlock(name="job1", agent=job_agent1),
        ChainAgentBlock(name="job2", agent=job_agent2),
        ChainAgentBlock(name="job_comparison", agent=job_comparison_agent),
    ],
)

In [40]:
output = chain.invoke({"job1": job1, "job2": job2})

02/08/24 19:35:02 INFO Running Block: job1
02/08/24 19:35:02 INFO Prompt:
Job description:
ELECTRICAL ENGINEERING ASSOCIATE
Class Code:       7525
Open Date:  09-30-16
REVISED: 10-04-16
 (Exam Open to All, including Current City Employees)
ANNUAL SALARY 

$66,231 to $94,252; $74,082 to $105,444; $82,497 to $117,346; and $89,638 to $127,556
The salary in the Department of Water and Power is $77,360 to $96,110; $91,934 to $114,213; $99,722 to $123,881; and $107,156 to 
$133,130

NOTES:

1. Candidates from the eligible list are normally appointed to vacancies in the lower pay grade positions.
2. For information regarding reciprocity between City of Los Angeles departments and LADWP, go to: http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.
3. The current salary range is subject to change. You may confirm the starting salary with the hiring department before accepting a job offer.

DUTIES

An Electrical Engineering Associate performs professional electrical engineering work in the pr

In [41]:
pprint(output.output.similarities)

('Both jobs are in the field of electrical work and involve dealing with '
 'electrical systems and equipment. They both may also involve work in various '
 'facilities and buildings.')


In [42]:
pprint(output.output.differences)

('The Electrical Engineering Associate is more focused on design, planning, '
 'and quality assurance, while the Electrical Mechanic is more hands-on, '
 'dealing with the installation and maintenance of electrical circuits and '
 'equipment. The Electrical Engineering Associate may also perform code '
 'enforcement functions and provide technical direction. The Electrical '
 'Mechanic may work at heights, near hazardous materials, and in confined '
 'spaces. The salary range also varies, with the Electrical Engineering '
 'Associate having a wider range and potentially higher pay.')


### Machine Learning Code Generation

Consider using an AI agent to generate machine learning code. Initially, you might think to give the LLM a piece of the dataset to generate the code. But this simple idea might not work well. The LLM could make mistakes, like treating a classification task as a regression because of how the classes are represented numerically.

To fix this, you could have the LLM figure out the problem type first, using insights from the dataset and prompts from the user, before generating any code. While this sounds doable with just one agent, it actually makes things more complicated when it comes to formatting the output. Sometimes, the LLM will output the code along with a description of the problem type and other information, while other times, it will solely respond with the code.

To make things simpler, it's better to split the AI agent into two parts that work together. The 1st LLM looks at the dataset and problem description to figure out what kind of machine learning problem it is, like classification. Then, the 2nd LLM uses this info to generate machine learning code. This split not only makes things easier for the LLMs but also helps humans understand the structure more easily. It also simplifies the process of adjusting the prompts to ensure that the LLM only responds with code.

| Input |
|------|
| problem description |
| dataset size |
| dataset schema |
| dataset snippet |

<br/>

| LLM that determines the machine learning problem |
|------|
| `Input` problem description, dataset size, dataset schema |
| `Output` modeling problem |

<br/>

| LLM that generates machine learning code |
|------|
| `Input` modeling problem, dataset size, dataset schema, dataset snippet |
| `Output` code |

<br/>

| Output |
|------|
| code |

In [43]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

Respond with 1 of the following machine learning problems:
- Classification
- Regression
- Clustering
- Time series forecasting"""

prompt = """Choose the machine learning problem best suited for the following problem and dataset.

Problem description:
{problem_description}

Dataset:
Number of rows: {dataset_size}
Schema:
{dataset_schema}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=250,
    temperature=0.2,
)

problem_finder_agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt=prompt,
    prompt_variables=["problem_description", "dataset_size", "dataset_schema"],
    tools=None,
    output_type=OutputType.STRING,
    iterations=3,
)

In [44]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

You can solve machine learning problems for:
- Classification
- Regression
- Clustering
- Time series forecasting

You have access to the following Python libraries:
- pandas
- numpy
- scikit-learn"""

prompt = """Given the following machine learning problem, respond with Python code.

Modeling problem: {modeling_problem}

Dataset:
Number of rows: {dataset_size}
Schema:
{dataset_schema}
First 10 rows of dataset:
{dataset_snippet}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1000,
    temperature=0.2,
)

ml_agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt=prompt,
    prompt_variables=["modeling_problem", "dataset_size", "dataset_schema", "dataset_snippet"],
    tools=None,
    output_type=OutputType.STRING,
    iterations=3,
)

In [45]:
class MLCodeGeneration(BaseModel):
    problem_description: str = Field(description="The user problem")
    dataset_size: int = Field(description="Size of the dataset")
    dataset_schema: str = Field(description="Schema of the dataset")
    dataset_snippet: str = Field(description="Snippet of the dataset")

ml_chain = Chain(
    name="Generate Machine Learning Code",
    description="Allows you to generate ML code",
    inputs=MLCodeGeneration,
    output="code",
    blocks=[ChainAgentBlock(name="modeling_problem", agent=problem_finder_agent), ChainAgentBlock(name="code", agent=ml_agent)],
)

In [46]:
info_str = StringIO()
df_tweets.info(buf=info_str)
dataset_schema = info_str.getvalue()

In [47]:
output = ml_chain.invoke({
    "problem_description": "I want to classify the sentiment of tweets.",
    "dataset_size": len(df_tweets),
    "dataset_schema": dataset_schema,
    "dataset_snippet": str(df_tweets.head(10).to_markdown())
})

02/08/24 19:35:34 INFO Running Block: modeling_problem
02/08/24 19:35:34 INFO Prompt:
Choose the machine learning problem best suited for the following problem and dataset.

Problem description:
I want to classify the sentiment of tweets.

Dataset:
Number of rows: 1600000
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

02/08/24 19:35:38 INFO Raw Output:
Thought: The problem description is about classifying the sentiment of tweets. This is a clear indication of a classification problem. The dataset also supports this as it has a 'sentiment' co

In [48]:
print(output.output)

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Load the data
# df = pd.read_csv('data.csv')

# Preprocess the text data
df['tweet'] = df['tweet'].str.lower()
df['tweet'] = df['tweet'].str.replace('[^\w\s]','')
df['tweet'] = df['tweet'].apply(lambda x: ' '.join(word for word in x.split() if word not in (stopwords.words('english'))))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['sentiment'], test_size=0.2, random_state=42)

# Convert the text data into a matrix of token counts
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

# Train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_counts, y_train)

# Make predictions on the testing set
y_pre

### Forecasting

Structured output formats offer another advantage: when chaining blocks together, we're not limited to chaining LLMs only; we can also use any arbitrary function. Since we know the fields the LLM will output, we can simply pass those to run a function that takes these arguments. In this example, we let an LLM extract the region and use the output to run a function for time series forecasting. In practice, it is also common to have LLMs integrate with forecasting models via tools.

| Input |
|------|
| question |

<br/>

| LLM that determines the region |
|------|
| `Input` question |
| `Output` region |

<br/>

| Code block that does the forecasting |
|------|
| `Input` region |
| `Output` forecast |

<br/>

| Output |
|------|
| forecast |

In [49]:
class Forecast(BaseModel):
    region: str = Field(description="A valid region that the model can perform forecasts for")

def forecast_earthquakes(region: str) -> dict[str, list]:
    if not region:
        return
    df = get_forecast(region)
    today = pd.Timestamp.now()
    df = df.loc[df.Date > today]
    df = df[["Date", "Magnitude Forecast", "Depth Forecast"]]
    return df.to_dict(orient="records")

In [50]:
get_regions_tool = Tool(
    function=get_regions,
    name="Find Regions",
    description="Use this tool to access the available regions that can be used for forecasting."
)

In [51]:
prompt = """{question}

Use the Find Regions tool to determine the name of the region the user is interested in.

Only respond with 1 word: the name of the region."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=250,
    temperature=0.2,
)

region_finder_agent = Agent.create(
    llm=llm,
    system_prompt="",
    prompt=prompt,
    prompt_variables=["question"],
    tools=[get_regions_tool],
    output_type=OutputType.STRING,
    iterations=3,
)

In [52]:
class ForecastQuery(BaseModel):
    question: str = Field(description="The user query")

forecast_chain = Chain(
    name="Forecast Earthquakes",
    description="Allows you to run earthquake forecast",
    inputs=ForecastQuery,
    output="forecast",
    blocks=[
        ChainAgentBlock(name="region", agent=region_finder_agent),
        ChainToolBlock(name="forecast", function=forecast_earthquakes, inputs=Forecast),
    ],
)

In [53]:
output = forecast_chain.invoke({"question": "Run a forecast for California."})

02/08/24 19:35:55 INFO Running Block: region
02/08/24 19:35:55 INFO Prompt:
Run a forecast for California.

Use the Find Regions tool to determine the name of the region the user is interested in.

Only respond with 1 word: the name of the region.
02/08/24 19:35:58 INFO Raw Output:
Thought: The user wants to run a forecast for California. I need to use the Find Regions tool to determine the exact name of the region.

Tool: Find Regions

Tool Input: {}
02/08/24 19:35:58 INFO Thought:
The user wants to run a forecast for California. I need to use the Find Regions tool to determine the exact name of the region.
02/08/24 19:35:58 INFO Tool:
Find Regions
02/08/24 19:35:58 INFO Tool Input:
{}
02/08/24 19:36:01 INFO Tool Response:
{'Idaho', 'Turkey', 'Washington', 'Greece', 'Alaska', 'Puerto Rico', 'Italy', 'Chile', 'Tonga', 'Papua New Guinea', 'Indonesia', 'Nevada', 'Oregon', 'Baja California', 'Oklahoma', 'Hawaii', 'California', 'Japan', 'Wyoming', 'Philippines', 'Russia', 'Mexico', 'Montan

In [54]:
print(output.output)

[{'Date': Timestamp('2024-08-03 00:00:00'), 'Magnitude Forecast': 1.6627185662399055, 'Depth Forecast': 5.410133555324581}, {'Date': Timestamp('2024-08-04 00:00:00'), 'Magnitude Forecast': 1.6417918556702367, 'Depth Forecast': 5.636850019032636}, {'Date': Timestamp('2024-08-05 00:00:00'), 'Magnitude Forecast': 1.7877133771613778, 'Depth Forecast': 7.268750830707475}]


### LLM-powered Functions

AI agents can collaborate with each other in a chain-like fashion, as demonstrated earlier. Another approach involves embedding an AI agent within a function that receives arguments, turning it into a tool like any other function in code. The true strength of LLMs lies in their ability to use tools. By leveraging this capability, we can construct a network of interconnected functionalities, where LLMs utilize tools that are powered by LLMs.

This understanding allows us to develop AI agents capable of handling diverse tasks. The following example illustrates this concept. While the combination of domains may seem mismatched in this context, we can repurpose the previously created agents and chains into a tool, which we then integrate into an LLM. This enables the LLM to answer multi-domain questions effectively.

In [55]:
class EarthquakeQuery(BaseModel):
    question: str = Field(description="The earthquake related user query")

earthquake_chain = Chain(
    name="Earthquake Agent",
    description="Allows you to answer earthquake related questions",
    inputs=EarthquakeQuery,
    output="earthquake",
    blocks=[ChainAgentBlock(name="earthquake", agent=earthquake_agent)]
)

In [56]:
system_prompt = """You are an Agent that delegates tasks to other Agents by using the appropriate tools.

Use the Earthquake Agent when the question is about earthquakes.

Use the ML Agent when the user wants you to generate machine learning code."""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4-32k',
    max_tokens=2000,
    temperature=0.2,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

almighty_agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{prompt}",
    prompt_variables=["prompt"],
    tools=[ml_chain.as_tool(), earthquake_chain.as_tool()],
    output_type=OutputType.STRING,
    iterations=3,
)

In [57]:
output = almighty_agent.invoke({"prompt": "How many earthquakes happened today?"})

02/08/24 19:36:04 INFO Prompt:
How many earthquakes happened today?
02/08/24 19:36:08 INFO Raw Output:
Thought: The user is asking about earthquakes, so I should use the Earthquake Agent tool to answer this question.

Tool: Earthquake Agent

Tool Input: {"question": "How many earthquakes happened today?"}
02/08/24 19:36:08 INFO Thought:
The user is asking about earthquakes, so I should use the Earthquake Agent tool to answer this question.
02/08/24 19:36:08 INFO Tool:
Earthquake Agent
02/08/24 19:36:08 INFO Tool Input:
{'question': 'How many earthquakes happened today?'}
02/08/24 19:36:08 INFO Running Block: earthquake
02/08/24 19:36:08 INFO Prompt:
How many earthquakes happened today?
02/08/24 19:36:13 INFO Raw Output:
Thought: To answer this question, I need to count the number of earthquakes that occurred today. I will use the "Current Date" tool to get today's date and then use the "Count Earthquakes" tool with the start time set to the beginning of today and the end time set to th

In [58]:
print(output.final_answer)

There have been 156 earthquakes today.


In [59]:
almighty_agent.chat.reset()

In [60]:
prompt = f"""Respond with the machine learning code that trains a model to classify the sentiment of tweets.

Dataset:
Number of rows: {len(df_tweets)}
Schema:
{dataset_schema}
First 10 rows of dataset:
{df_tweets.head(10).to_markdown()}"""

output = almighty_agent.invoke({"prompt": prompt})

02/08/24 19:36:46 INFO Prompt:
Respond with the machine learning code that trains a model to classify the sentiment of tweets.

Dataset:
Number of rows: 1600000
Schema:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

First 10 rows of dataset:
|    |   sentiment |         id | date                         | query    | user            | tweet                                                                                                               |
|---:|------------:|-----------:|:-----------------------------|:---------|:----------------|:-------

In [61]:
print(output.final_answer)

Here is the Python code for training a sentiment analysis model on your dataset:

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Assuming that df is your DataFrame
# Preprocess the text data
df['tweet'] = df['tweet'].str.lower()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['sentiment'], test_size=0.2, random_state=42)

# Convert the text data into a matrix of token counts
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

# Train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_counts, y_train)

# Make predictions on the testing set
y_pred = clf.predict(X_test_counts)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y